In [2]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date
import json


with open("login.json", "r") as j:
    misDatos = json.load(j)
#para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
#para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

SalesforceAuthenticationFailed: INVALID_LOGIN: Nombre de usuario, contraseña, token de seguridad o usuario no válidos o bloqueados.

In [ ]:
results = sf.query_all(
    """
    select Id, IdentificationId__pc from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
    }
)
Pacientes_SF.shape



SalesforceExpiredSession: Expired session for https://careassistance.my.salesforce.com/services/data/v42.0/query/?q=%0A++++select+Id%2C+IdentificationId__pc+from+Account+where+ispersonaccount+%3D+True%0A+++. Response content: [{'message': 'Your password has expired. Please reset your password.', 'errorCode': 'INVALID_OPERATION_WITH_EXPIRED_PASSWORD'}]

In [ ]:
results = sf.query_all(
    """
    select id,IdentificationId__c from Contact 
   """
)
Pacientes_SF2 = pd.DataFrame(results["records"])

Pacientes_SF2 = Pacientes_SF2[
    [
        "Id",
        "IdentificationId__c",
    ]
]
Pacientes_SF2 = Pacientes_SF2.rename(
    columns={
        "Id": "Id Contact Paciente",
        "IdentificationId__c": "Rut",
    }
)
Pacientes_SF2.shape

(1857574, 2)

In [ ]:
Pacientes_SF2 = pd.merge(
    left=Pacientes_SF,
    right=Pacientes_SF2,
    how="left",
    left_on="Rut",
    right_on="Rut",
)
Pacientes_SF2.shape

(1857574, 3)

In [ ]:
Pacientes_SF2.head()

,Id Account Paciente,Rut,Id Contact Paciente
0,0018c00002amLTMAA2,173805004,0038c000035CV93AAG
1,0018c00002amLUGAA2,153338949,0038c000035CV94AAG
2,0018c00002amLUHAA2,177735213,0038c000035CV95AAG
3,0018c00002amLUIAA2,169332924,0038c000035CV9zAAG
4,0018c00002amLUJAA2,172957692,0038c000035CVA0AAO


In [ ]:
results = sf.query_all(
    """
    select id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c where Programas_por_Empresas__r.Empresa__r.Estado__c <> 'Dado De Baja - Sin Servicio' 
   """
)
Paciente_Programa = pd.DataFrame(results["records"])

Paciente_Programa = Paciente_Programa[
    [
        "Paciente__c",
        "Programas_por_Empresas__c",
    ]
]
Paciente_Programa = Paciente_Programa.rename(
    columns={
        "Paciente__c": "Id paciente account",
        "Programas_por_Empresas__c": "Programas por empresa",
    }
)
Paciente_Programa.shape

(1306171, 2)

In [ ]:
Paciente_Programa.head()

,Id paciente account,Programas por empresa
0,0018c00002bKcB9AAK,a1W8c000007vDN8EAM
1,0018c00002bKcBAAA0,a1W8c000007vDN8EAM
2,0018c00002bKcBaAAK,a1W8c000007vDN8EAM
3,0018c00002bKcBBAA0,a1W8c000007vDN8EAM
4,0018c00002bKcBbAAK,a1W8c000007vDN8EAM


In [ ]:
results = sf.query_all(
    """
    select Id,Empresa__c from Programas_por_Empresas__c  
   """
)
empresas = pd.DataFrame(results["records"])

empresas = empresas[
    [
        "Id",
        "Empresa__c",
    ]
]
empresas = empresas.rename(
    columns={
        "Id": "Programas_por_Empresas__c",
        "Empresa__c": "Id Account Empresa",
    }
)
empresas.shape





(7479, 2)

In [ ]:
empresas.shape

(7479, 2)

In [ ]:
Paciente_Programa = pd.merge(
    left=Paciente_Programa,
    right=empresas,
    how="left",
    left_on="Programas por empresa",
    right_on="Programas_por_Empresas__c",
)
Paciente_Programa.shape

(1306171, 4)

In [ ]:
Paciente_Programa.head()

,Id paciente account,Programas por empresa,Programas_por_Empresas__c,Id Account Empresa
0,0018c00002bKcB9AAK,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM
1,0018c00002bKcBAAA0,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM
2,0018c00002bKcBaAAK,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM
3,0018c00002bKcBBAA0,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM
4,0018c00002bKcBbAAK,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM


In [ ]:
results = sf.query_all(
    """
    select AccountId, ContactId from AccountContactRelation 
   """
)
accountcontact = pd.DataFrame(results["records"])

accountcontact = accountcontact[
    [
        "AccountId",
        "ContactId",
    ]
]
accountcontact = accountcontact.rename(
    columns={
        "AccountId": "Id account",
        "ContactId": "id contacto",
    }
)
accountcontact.shape

(1421262, 2)

In [ ]:
accountcontact['LLave'] = accountcontact['id contacto'] + accountcontact['Id account'] 
accountcontact.head()

,Id account,id contacto,LLave
0,0018c00002axQdlAAE,0038c000036jelbAAA,0038c000036jelbAAA0018c00002axQdlAAE
1,0018c00002axQdlAAE,0038c000035CWLYAA4,0038c000035CWLYAA40018c00002axQdlAAE
2,0018c00002axQdlAAE,0038c000035CaxiAAC,0038c000035CaxiAAC0018c00002axQdlAAE
3,0018c00002axQdlAAE,0038c000036jelcAAA,0038c000036jelcAAA0018c00002axQdlAAE
4,0018c00002axQa5AAE,0038c000036jLDoAAM,0038c000036jLDoAAM0018c00002axQa5AAE


In [ ]:
accountcontact.shape

(1421262, 3)

In [ ]:
Paciente_Programa.head()

,Id paciente account,Programas por empresa,Programas_por_Empresas__c,Id Account Empresa
0,0018c00002bKcB9AAK,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM
1,0018c00002bKcBAAA0,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM
2,0018c00002bKcBaAAK,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM
3,0018c00002bKcBBAA0,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM
4,0018c00002bKcBbAAK,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM


In [ ]:
Paciente_Programa.shape

(1306171, 4)

In [ ]:
Pacientes_SF2.head()

,Id Account Paciente,Rut,Id Contact Paciente
0,0018c00002amLTMAA2,173805004,0038c000035CV93AAG
1,0018c00002amLUGAA2,153338949,0038c000035CV94AAG
2,0018c00002amLUHAA2,177735213,0038c000035CV95AAG
3,0018c00002amLUIAA2,169332924,0038c000035CV9zAAG
4,0018c00002amLUJAA2,172957692,0038c000035CVA0AAO


In [ ]:
Pacientes_SF2.shape

(1857574, 3)

In [ ]:
programa_paciente = pd.merge(
    left=Paciente_Programa,
    right=Pacientes_SF2,
    how="left",
    left_on="Id paciente account",
    right_on="Id Account Paciente",
)
programa_paciente.shape

(1306171, 7)

In [ ]:
programa_paciente.head()

,Id paciente account,Programas por empresa,Programas_por_Empresas__c,Id Account Empresa,Id Account Paciente,Rut,Id Contact Paciente
0,0018c00002bKcB9AAK,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM,0018c00002bKcB9AAK,21868386K,0038c000035vzprAAA
1,0018c00002bKcBAAA0,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM,0018c00002bKcBAAA0,107863311,0038c000035vzpsAAA
2,0018c00002bKcBaAAK,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM,0018c00002bKcBaAAK,119689163,0038c000035vzqIAAQ
3,0018c00002bKcBBAA0,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM,0018c00002bKcBBAA0,108859369,0038c000035vzptAAA
4,0018c00002bKcBbAAK,a1W8c000007vDN8EAM,a1W8c000007vDN8EAM,0018c00002axQXrAAM,0018c00002bKcBbAAK,136554638,0038c000035vzqJAAQ


In [ ]:
programa_paciente.shape

(1306171, 7)

In [ ]:
programa_paciente = programa_paciente[
    [
        "Id Contact Paciente",
        "Id Account Empresa",
    ]
]
programa_paciente['LLave'] = programa_paciente['Id Contact Paciente'] + programa_paciente['Id Account Empresa']
programa_paciente = programa_paciente.rename(
    columns={
        "Id Contact Paciente": "ContactId",
        "Id Account Empresa": "AccountId",
    }
)
programa_paciente.head()

,ContactId,AccountId,LLave
0,0038c000035vzprAAA,0018c00002axQXrAAM,0038c000035vzprAAA0018c00002axQXrAAM
1,0038c000035vzpsAAA,0018c00002axQXrAAM,0038c000035vzpsAAA0018c00002axQXrAAM
2,0038c000035vzqIAAQ,0018c00002axQXrAAM,0038c000035vzqIAAQ0018c00002axQXrAAM
3,0038c000035vzptAAA,0018c00002axQXrAAM,0038c000035vzptAAA0018c00002axQXrAAM
4,0038c000035vzqJAAQ,0018c00002axQXrAAM,0038c000035vzqJAAQ0018c00002axQXrAAM


In [ ]:
programa_paciente.shape

(1306171, 3)

In [ ]:
programa_paciente["RutEstaoNo"] = programa_paciente["LLave"].isin(accountcontact['LLave'])
programa_paciente.shape

(1306171, 4)

In [ ]:
programa_paciente.head()


,ContactId,AccountId,LLave,RutEstaoNo
0,0038c000035vzprAAA,0018c00002axQXrAAM,0038c000035vzprAAA0018c00002axQXrAAM,True
1,0038c000035vzpsAAA,0018c00002axQXrAAM,0038c000035vzpsAAA0018c00002axQXrAAM,True
2,0038c000035vzqIAAQ,0018c00002axQXrAAM,0038c000035vzqIAAQ0018c00002axQXrAAM,True
3,0038c000035vzptAAA,0018c00002axQXrAAM,0038c000035vzptAAA0018c00002axQXrAAM,True
4,0038c000035vzqJAAQ,0018c00002axQXrAAM,0038c000035vzqJAAQ0018c00002axQXrAAM,True


In [ ]:
programa_paciente1 = programa_paciente[programa_paciente["RutEstaoNo"] == False] 

In [ ]:
accountcontact["RutEstaoNo"] = accountcontact["LLave"].isin(programa_paciente['LLave'])
accountcontact1 = accountcontact[accountcontact["RutEstaoNo"] == False]
accountcontact1.head()


,Id account,id contacto,LLave,RutEstaoNo
23,0018c00002axQa5AAE,0038c000036jMBzAAM,0038c000036jMBzAAM0018c00002axQa5AAE,False
58,0018c00002axQdeAAE,0038c000036jPGcAAM,0038c000036jPGcAAM0018c00002axQdeAAE,False
149,0018c00002axQdcAAE,0038c000036izTKAAY,0038c000036izTKAAY0018c00002axQdcAAE,False
201,0018c00002axQdcAAE,0038c000036izU9AAI,0038c000036izU9AAI0018c00002axQdcAAE,False
538,0018c00002axQdcAAE,0038c000036izlMAAQ,0038c000036izlMAAQ0018c00002axQdcAAE,False


In [ ]:
accountcontact.shape

(1421262, 4)

In [ ]:
programa_paciente1 = programa_paciente1.drop_duplicates(['LLave'], keep='last')
programa_paciente1.shape

(851, 4)

In [ ]:
programa_paciente1.to_excel("F:/Empresas SF/reportes/accountcontact.xlsx", index=False)
#accountcontact1.to_excel("F:/Empresas SF/reportes/accountcontact1.xlsx", index=False)